Housing EDA

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
#import ml_insights as mli

from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

from sklearn.linear_model import LinearRegression, Lasso, LassoCV

In [94]:
%matplotlib inline

# Load Data and basic EDA

In [95]:
## Load in the Ames Housing Data
datafile = "data/AmesHousing.csv"
df=pd.read_csv(datafile)

Basic EDA

In [96]:
# Shape

df.shape

(2930, 82)

Remove spaces from column names

In [97]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

df.columns

Index(['order', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'alley', 'lot_shape', 'land_contour', 'utilities',
       'lot_config', 'land_slope', 'neighborhood', 'condition_1',
       'condition_2', 'bldg_type', 'house_style', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'roof_style',
       'roof_matl', 'exterior_1st', 'exterior_2nd', 'mas_vnr_type',
       'mas_vnr_area', 'exter_qual', 'exter_cond', 'foundation', 'bsmt_qual',
       'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1',
       'bsmtfin_type_2', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating', 'heating_qc', 'central_air', 'electrical', '1st_flr_sf',
       '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'fireplace_qu', 'garage_type', 'garage_yr_blt',
      

Numeric and non-numeric columns

In [98]:
df.dtypes

order               int64
pid                 int64
ms_subclass         int64
ms_zoning          object
lot_frontage      float64
                   ...   
mo_sold             int64
yr_sold             int64
sale_type          object
sale_condition     object
saleprice           int64
Length: 82, dtype: object

In [99]:
numeric_columns = df.select_dtypes(include=[np.number]).columns
num_numeric_columns = len(numeric_columns)

non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns
num_non_numeric_columns = len(non_numeric_columns)

print(f"Number of numeric columns: {num_numeric_columns}")
print(f"Number of non-numeric columns: {num_non_numeric_columns}")

Number of numeric columns: 39
Number of non-numeric columns: 43


Looking at nulls

In [100]:
df.isna().any()

order             False
pid               False
ms_subclass       False
ms_zoning         False
lot_frontage       True
                  ...  
mo_sold           False
yr_sold           False
sale_type         False
sale_condition    False
saleprice         False
Length: 82, dtype: bool

In [101]:
df.columns[df.isna().any()]

Index(['lot_frontage', 'alley', 'mas_vnr_type', 'mas_vnr_area', 'bsmt_qual',
       'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1',
       'bsmtfin_type_2', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'electrical', 'bsmt_full_bath', 'bsmt_half_bath', 'fireplace_qu',
       'garage_type', 'garage_yr_blt', 'garage_finish', 'garage_cars',
       'garage_area', 'garage_qual', 'garage_cond', 'pool_qc', 'fence',
       'misc_feature'],
      dtype='object')

In [102]:
df.isna().sum().sort_values(ascending=False).head(20)

pool_qc           2917
misc_feature      2824
alley             2732
fence             2358
mas_vnr_type      1775
fireplace_qu      1422
lot_frontage       490
garage_cond        159
garage_finish      159
garage_yr_blt      159
garage_qual        159
garage_type        157
bsmt_exposure       83
bsmtfin_type_2      81
bsmt_qual           80
bsmt_cond           80
bsmtfin_type_1      80
mas_vnr_area        23
bsmt_full_bath       2
bsmt_half_bath       2
dtype: int64

Fill nulls

In [103]:
# Most nulls look like features some houses don't have. We can fill these with 0s or "None" depending on the feature.

df[numeric_columns] = df[numeric_columns].fillna(0)
df[non_numeric_columns] = df[non_numeric_columns].fillna('None')

View single record

In [104]:
df.head(1)

,order,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice
0,1,526301100,20,RL,141.0,31770,Pave,None,IR1,Lvl,...,0,None,None,None,0,5,2010,WD,Normal,215000


In [105]:
df.iloc[0]

order                     1
pid               526301100
ms_subclass              20
ms_zoning                RL
lot_frontage          141.0
                    ...    
mo_sold                   5
yr_sold                2010
sale_type               WD 
sale_condition       Normal
saleprice            215000
Name: 0, Length: 82, dtype: object

View individual variables

In [106]:
df['lot_frontage'].describe()

count    2930.000000
mean       57.647782
std        33.499441
min         0.000000
25%        43.000000
50%        63.000000
75%        78.000000
max       313.000000
Name: lot_frontage, dtype: float64

In [107]:
df['condition_1'].value_counts() # looks like ordinal

condition_1
Norm      2522
Feedr      164
Artery      92
RRAn        50
PosN        39
RRAe        28
PosA        20
RRNn         9
RRNe         6
Name: count, dtype: int64

In [108]:
df['condition_1'].unique()

array(['Norm', 'Feedr', 'PosN', 'RRNe', 'RRAe', 'Artery', 'PosA', 'RRAn',
       'RRNn'], dtype=object)

In [109]:
df['year_built'].value_counts()

year_built
2005    142
2006    138
2007    109
2004     99
2003     88
       ... 
1882      1
1898      1
1907      1
1875      1
1913      1
Name: count, Length: 118, dtype: int64

In [110]:
df['year_remod/add'].value_counts()

year_remod/add
1950    361
2006    202
2007    164
2005    141
2004    111
       ... 
1986     13
1981     13
2010     13
1983     11
1982      9
Name: count, Length: 61, dtype: int64

In [111]:
df['garage_area'].value_counts()

garage_area
0.0      158
576.0     97
440.0     96
484.0     76
240.0     69
        ... 
715.0      1
902.0      1
595.0      1
581.0      1
405.0      1
Name: count, Length: 603, dtype: int64

In [112]:
df['total_bsmt_sf'].value_counts()

total_bsmt_sf
0.0       80
864.0     74
672.0     29
912.0     26
1040.0    25
          ..
839.0      1
1146.0     1
1415.0     1
1684.0     1
1003.0     1
Name: count, Length: 1058, dtype: int64

Export cleaned df

In [114]:
df.to_csv("data/AmesHousingCleaned.csv", index=False)